In [1]:
import minsearch


In [2]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [4]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [5]:
search_results = search('I just discovered the course. Can I join now?')
print(search_results[0]['text'])

Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


In [6]:
from google import genai
import os
from google.genai import types

gemini_client = genai.Client()
def llm(prompt):
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash", 
        contents=prompt
    )
    response = response.text.replace("```json","").replace("```", "")

    return response

In [7]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [8]:
question = 'I just discovered the course. Can I join now?'

In [9]:
prompt = build_prompt(question, search_results)

In [10]:
print(llm(prompt))

Yes, you can join the course even if you don't register. You're still eligible to submit the homework.



In [11]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [12]:
print(rag(question))

Yes, even if you don't register, you're still eligible to submit the homeworks.



In [13]:
rag('How do I run Kafka in Docker?')

"If you encounter a `kafka.errors.NoBrokersAvailable: NoBrokersAvailable` error, it's likely that your Kafka broker Docker container isn't working. You can use `docker ps` to confirm. Then, navigate to the Docker Compose YAML file folder and run `docker compose up -d` to start all the instances.\n"

In [14]:
rag('How do I patch KDE under FreeBSD?')

'I am sorry, but I cannot answer this question because the provided context is empty.\n'

In [15]:
llm('How do I patch KDE under FreeBSD?')

'Patching KDE under FreeBSD can be done in a few ways, depending on what you want to patch. Here\'s a breakdown of the common scenarios and how to approach them:\n\n**1. Patching a Port (Recommended for Most Users)**\n\nThis is the most common and generally recommended method. It\'s suitable for applying patches to KDE components installed via the FreeBSD Ports Collection.\n\n*   **Identify the Port:** First, identify the specific KDE port you need to patch.  Let\'s say you need to patch `kde-frameworks/kio`.\n\n*   **Find the Port Directory:** Navigate to the port\'s directory within the Ports Collection.  This is typically under `/usr/ports`.\n\n    bash\n    cd /usr/ports/kde-frameworks/kio\n    \n\n*   **Obtain the Patch:** Get the patch file.  This might come from:\n    *   A bug report on the KDE bug tracker (bugs.kde.org)\n    *   A mailing list related to FreeBSD ports or KDE\n    *   From the FreeBSD Bugzilla\n    *   Directly from the upstream KDE developers\n    *   Your own

In [16]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

return response in JSON format.
""".strip()

In [17]:
question = "How can I run Docker on Windows 10?"
context = 'EMPTY'
prompt = prompt_template.format(question=question, context=context)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
How can I run Docker on Windows 10?
</QUESTION>

<CONTEXT> 
EMPTY
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}

If you can answer the QUESTION using CONTEXT, use this template:

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}

If the context doesn't contain the answer, use your own knowledge to answer the question

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}

return response in JSON format.


In [18]:
answer = llm(prompt)

In [19]:
print(answer)


{
"action": "ANSWER",
"answer": "You have a few options for running Docker on Windows 10:\n\n1.  **Docker Desktop:** This is the recommended and most common way. Docker Desktop uses WSL 2 (Windows Subsystem for Linux version 2) to run a Linux kernel, which then runs the Docker daemon. This provides excellent performance and integration with Windows.\n\n2.  **Docker Toolbox (Legacy):** This is an older method that uses VirtualBox to create a virtual machine running Linux, where Docker runs. Docker Toolbox is generally only recommended for older systems that don't support WSL 2, or for specific edge cases. Docker Desktop is generally preferred.\n\nHere's a bit more detail on each:\n\n*   **Docker Desktop (Recommended):**\n    *   **Prerequisites:** Windows 10 64-bit: Home or Pro 2004 (build 19041) or higher, or Enterprise or Education 1703 (build 15063) or higher. Enable WSL 2.\n    *   **Installation:** Download and install Docker Desktop from the Docker website. During installation, i

In [20]:
question = "Can I still join the course?"
context = 'EMPTY'
prompt = prompt_template.format(question=question, context=context)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
Can I still join the course?
</QUESTION>

<CONTEXT> 
EMPTY
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}

If you can answer the QUESTION using CONTEXT, use this template:

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}

If the context doesn't contain the answer, use your own knowledge to answer the question

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}

return response in JSON format.


In [21]:
answer_json = llm(prompt)
print(answer_json)


{
  "action": "SEARCH",
  "reasoning": "Since the context is empty, I need to search for information about course enrollment policies to determine if the student can still join."
}



In [22]:
import json

In [23]:
json.loads(answer_json)

{'action': 'SEARCH',
 'reasoning': 'Since the context is empty, I need to search for information about course enrollment policies to determine if the student can still join.'}

In [24]:
answer_json = llm(prompt)
print(answer_json)


{
"action": "SEARCH",
"reasoning": "Since the context is empty, I need to consult the FAQ or course information to determine if it's still possible to join the course. I don't have enough information to answer this question without further investigation."
}



In [25]:
answer_json = json.loads(answer_json)

In [26]:
answer_json['action']


def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    return context.strip()

In [27]:
search_results = search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
Can I still join the course?
</QUESTION>

<CONTEXT> 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Certificate - Can I follow the course in a self-paced mode and get a certificate?
answer: No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the cour

In [28]:
answer_json= llm(prompt)

In [29]:
print(answer_json)


{
  "action": "ANSWER",
  "answer": "Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  "source": "CONTEXT"
}



In [30]:
def agentic_rag_v1(question):
    context = "EMPTY"
    prompt = prompt_template.format(question=question, context=context)
    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(answer)

    if answer['action'] == 'SEARCH':
        print('need to perform search...')
        search_results = search(question)
        context = build_context(search_results)
        
        prompt = prompt_template.format(question=question, context=context)
        answer_json = llm(prompt)
        answer = json.loads(answer_json)
        print(answer)

    return answer

In [31]:
print(agentic_rag_v1('Can I still join the course?'))

{'action': 'SEARCH', 'reasoning': "Since the context is empty, I need to find information about course enrollment policies. I'll search the FAQ to see if there's any information about joining the course after it has started."}
need to perform search...
{'action': 'ANSWER', 'answer': "Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'source': 'CONTEXT'}
{'action': 'ANSWER', 'answer': "Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'source': 'CONTEXT'}


In [60]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is built with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.


Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

Return response only in JSON format using the templates provided below. 
Do not return any other information. 
Do not use markdown.
Do not include any formating in your response (characters like newline character).

Output templates:

If you want to perform search, use this template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}


<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [61]:
question = 'How do I do well on module 1'
max_iterations = 3
iteration_number = 0
search_queries = []
search_results = []
previous_actions = []
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is built with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.


Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current iteration number: 0

In [62]:
answer_json = llm(prompt)

In [63]:
answer_json

'{\n"action": "SEARCH",\n"reasoning": "The question is very broad. I need to find information regarding specific aspects of Module 1 that can help a student do well. Keywords related to preparation, important concepts, common mistakes, and available resources for Module 1 could be useful.",\n"keywords": ["Module 1 preparation tips", "Module 1 important concepts", "Module 1 common mistakes", "Module 1 resources"]\n}\n'

In [64]:
answer_json = json.loads(answer_json)

In [65]:
keywords = answer_json['keywords']

In [66]:
for kw in keywords:
    search_queries.append(kw)
    sr = search(kw)
    search_results.extend(sr)

In [67]:
search_queries

['Module 1 preparation tips',
 'Module 1 important concepts',
 'Module 1 common mistakes',
 'Module 1 resources']

In [68]:
len(search_results)

20

In [69]:
def dedup(seq):
    seen = set()
    result = []
    for el in seq:
        _id = el['_id']
        if _id in seen:
            continue
        seen.add(_id)
        result.append(el)
    return result

In [70]:
search_results = dedup(search_results)

In [71]:
search_results

[{'text': 'You need to look for the Py4J file and note the version of the filename. Once you know the version, you can update the export command accordingly, this is how you check yours:\n` ls ${SPARK_HOME}/python/lib/ ` and then you add it in the export command, mine was:\nexport PYTHONPATH=”${SPARK_HOME}/python/lib/Py4J-0.10.9.5-src.zip:${PYTHONPATH}”\nMake sure that the version under `${SPARK_HOME}/python/lib/` matches the filename of py4j or you will encounter `ModuleNotFoundError: No module named \'py4j\'` while executing `import pyspark`.\nFor instance, if the file under `${SPARK_HOME}/python/lib/` was `py4j-0.10.9.3-src.zip`.\nThen the export PYTHONPATH statement above should be changed to `export PYTHONPATH="${SPARK_HOME}/python/lib/py4j-0.10.9.3-src.zip:$PYTHONPATH"` appropriately.\nAdditionally, you can check for the version of ‘py4j’ of the spark you’re using from here and update as mentioned above.\n~ Abhijit Chakraborty: Sometimes, even with adding the correct version of p

In [72]:
previous_actions.append(answer_json)

In [73]:
previous_actions 

[{'action': 'SEARCH',
  'reasoning': 'The question is very broad. I need to find information regarding specific aspects of Module 1 that can help a student do well. Keywords related to preparation, important concepts, common mistakes, and available resources for Module 1 could be useful.',
  'keywords': ['Module 1 preparation tips',
   'Module 1 important concepts',
   'Module 1 common mistakes',
   'Module 1 resources']}]

In [74]:
iteration_number = 1
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is built with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.


Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current iteration number: 1

In [77]:
answer_json = llm(prompt)

In [78]:
answer_json

'\n{\n"action": "SEARCH",\n"reasoning": "The context provided contains information about errors and solutions related to specific technologies covered in various modules, but it lacks general advice or specific content related to doing well in Module 1. I need to search for more general strategies and topics covered in Module 1 to provide a more helpful answer. The previous search focused on preparation tips, important concepts, common mistakes and resources. I will now refine the search by looking into the specific topics covered in module 1 such as \'Docker\' and \'Terraform\' and add \'best practices\' to the search.",\n"keywords": ["Module 1 Docker best practices", "Module 1 Terraform best practices", "Module 1 topics covered"]\n}\n'

In [79]:
answer_json = json.loads(answer_json.replace('\n', ''))
keywords = answer_json['keywords']
for kw in keywords:
    search_queries.append(kw)
    sr = search(kw)
    search_results.extend(sr)
search_results = dedup(search_results)
previous_actions.append(answer_json)

iteration_number = 2
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is built with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.


Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current iteration number: 2

In [80]:
answer_json = llm(prompt)

In [81]:
answer_json

'\n{\n"action": "ANSWER",\n"answer": "Module 1 focuses on Docker and Terraform. To do well, make sure you understand the core concepts of both technologies, including containerization, image building, infrastructure as code, and resource provisioning. Pay close attention to best practices for writing Dockerfiles and Terraform configurations, focusing on security, efficiency, and maintainability. Some common issues in Module 1 include problems with `psycopg2` in Python environments, which can often be resolved by installing or updating the package using `pip install psycopg2-binary`. Also, ensure you are running `terraform init` in the correct directory containing your Terraform configuration files to avoid errors. For Windows users, file system performance can be optimized by storing code within the default Linux distro when using Docker. If you encounter issues with Terraform and JWT tokens on WSL, synchronizing your system time with `sudo hwclock -s` might resolve the problem. Review

In [82]:
answer = json.loads(answer_json.replace('\n', ''))

In [83]:
print(answer['answer'])

Module 1 focuses on Docker and Terraform. To do well, make sure you understand the core concepts of both technologies, including containerization, image building, infrastructure as code, and resource provisioning. Pay close attention to best practices for writing Dockerfiles and Terraform configurations, focusing on security, efficiency, and maintainability. Some common issues in Module 1 include problems with `psycopg2` in Python environments, which can often be resolved by installing or updating the package using `pip install psycopg2-binary`. Also, ensure you are running `terraform init` in the correct directory containing your Terraform configuration files to avoid errors. For Windows users, file system performance can be optimized by storing code within the default Linux distro when using Docker. If you encounter issues with Terraform and JWT tokens on WSL, synchronizing your system time with `sudo hwclock -s` might resolve the problem. Review the provided course materials, and co

In [84]:
question = "what do I need to do to be successful at module 1?"

search_queries = []
search_results = []
previous_actions = []


iteration = 0

while True:
    print(f'ITERATION #{iteration}...')

    context = build_context(search_results)
    prompt = prompt_template.format(
        question=question,
        context=context,
        search_queries="\n".join(search_queries),
        previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
        max_iterations=3,
        iteration_number=iteration
    )

    print(prompt)

    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(json.dumps(answer, indent=2))

    previous_actions.append(answer)

    action = answer['action']
    if action != 'SEARCH':
        break

    keywords = answer['keywords']
    search_queries = list(set(search_queries) | set(keywords))
    
    for k in keywords:
        res = search(k)
        search_results.extend(res)

    search_results = dedup(search_results)
    
    iteration = iteration + 1
    if iteration >= 4:
        break

    print()

ITERATION #0...
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is built with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.


Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current ite

In [85]:
iteration

2

In [86]:
from google.genai import types

# Define the function declaration for the model
search_tool = {
    "name": "search",
    "description": "Searches the FAQ database.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ",
            }
        },
        "required": ["query"],
    },
}

tools = types.Tool(function_declarations=[search_tool])
config = types.GenerateContentConfig(tools=[tools])

def llm(prompt):
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash", 
        contents=prompt,
        config=config
    )
    return response

prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and your task is to answer it.

<QUESTION>
{question}
</QUESTION>
""".strip()

prompt = prompt_template.format(question=question)

response = llm(prompt)

# Check for a function call
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")
    #  In a real app, you would call your function here:
    #  result = schedule_meeting(**function_call.args)
else:
    print("No function call found in the response.")
    print(response.text)

Function to call: search
Arguments: {'query': 'how to succeed in module 1'}


In [87]:
function_call

FunctionCall(id=None, args={'query': 'how to succeed in module 1'}, name='search')

In [88]:
type(function_call)

google.genai.types.FunctionCall

In [89]:
function_call.name

'search'

In [90]:
f_name = function_call.name

In [91]:
arguments = function_call.args

In [92]:
arguments['query']

'how to succeed in module 1'

In [93]:
f = globals()[f_name]

In [94]:
results = f(**arguments)

In [95]:
# Create a function response part
function_response_part = types.Part.from_function_response(
    name=f_name,
    response={"result": results},
)
contents = [
    prompt
]

# Append function call and result of the function execution to contents
contents.append(response.candidates[0].content) # Append the content from the model's response.
contents.append(types.Content(role="user", parts=[function_response_part])) # Append the function response

final_response = gemini_client.models.generate_content(
    model="gemini-2.5-flash",
    config=config,
    contents=contents,
)

print(final_response.text)

I don't have information on general tips for success in Module 1. My search results for "how to succeed in module 1" returned specific technical solutions for common errors encountered in Module 1, such as issues with `psycopg2` or `SQLAlchemy`.

If you are encountering a specific problem or error in Module 1, please let me know and I'll try my best to help you troubleshoot it.


In [96]:
print(final_response)

candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=b'\n\xc9\x04\x01T\xa8\\\xee#,\x96\x8df\x18n\x1b\x14\xa3\xaf\x1e\xfd\x11c\xb96f\x8e\x05/\x8b\xea`\xa9\xa8\xe5\x16\x86c\xcaE\xca\xdc\x87\xef^\xef\x1e\xa8\xf2\xf0\x7fr\xe7L\x86Gf\x88+,\x8f\x12\xe8\xf5v\xd6\x95\x84\x84\xf9\xc2\xe0J<\x92>\x1ftV\xbc\xd0F\x9a:\x8c\xd9\xe7\x11\xad\x04\xfc\x0enH~d\x95\xc7\x0c\xa7\x16n\xb4\x05wyK\xca\xad]C\x80m\xd1\xbc\x17R-\xa3f\xb3?\xa1\xc9d$\xef\x10t\x01\x05s\xd78@\x027=\xcaMlV\x1f<\x8e\x9b\xb0\x16\xd6lu\x86\x92\xa9\xd1\xf9\x0c\xadpks\xf1-\x7f$\xab\x9c\r\xa8{\xc5\x0b\xf66\xc7A|>\x00\xb5Z\xcf\xb5H^\xedL1\xf7\xc6 \xe2\r\xfa\xd36\x18\xe0(\xf6\x1a^8nZ\xf8\x91.\x85\xd8\x0c\xf7@.&%\xdd@\xa0\xe2\x86\xfe\x98\n\x02\xba|\xfa\xfd\xfcs<QS\xca\x98<\x895a$\xa2LL\x9e\xee\xef\xc9;\x87\xda\xac\x12\xae\xc1\x0e+\x7fjP{u\x16v\xab\x1c\x9fh,\xe0\x93\x18 m\xdd!\x82@\xceX\x87\xac\'"\xfbQxbD\r|\xe4\xe8\xe5\xf1\xder\xc59\x12w#\xd06\x9b\xb4JWm\nJ\x98;

In [97]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and your task is to answer it.

If you look up something in FAQ, convert the student question into multiple queries.

<QUESTION>
{question}
</QUESTION>
""".strip()

prompt = prompt_template.format(question=question)

response = llm(prompt)

# Check for a function call
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")
    #  In a real app, you would call your function here:
    #  result = schedule_meeting(**function_call.args)
else:
    print("No function call found in the response.")
    print(response.text)





Function to call: search
Arguments: {'query': 'success module 1'}


In [98]:
function_calls = [part.function_call for part in response.candidates[0].content.parts]

In [99]:
function_calls

[FunctionCall(id=None, args={'query': 'success module 1'}, name='search'),
 FunctionCall(id=None, args={'query': 'module 1 tips'}, name='search'),
 FunctionCall(id=None, args={'query': 'module 1 requirements'}, name='search')]

In [100]:
contents = [
    prompt
]

def do_call(tool_call_response):
    function_name = tool_call_response.name
    arguments = tool_call_response.args

    f = globals()[function_name]
    result = f(**arguments)

    function_response_part = types.Part.from_function_response(
        name=f_name,
        response={"result": result},
    )

    return function_response_part

parts = []
for call in function_calls:
    function_response_part = do_call(call)
    parts.append(function_response_part)
    
# Append function call and result of the function execution to contents   
contents.append(response.candidates[0].content) # Append the content from the model's response.
contents.append(types.Content(role="user", parts=parts)) # Append the function response

In [101]:
final_response = gemini_client.models.generate_content(
    model="gemini-2.5-flash",
    config=config,
    contents=contents,
)

print(final_response.text)

I'm sorry, I couldn't find any specific information about general success strategies or requirements for Module 1 in the FAQ. The search results mainly provided solutions for technical issues encountered in Module 1, such as "ModuleNotFoundError: No module named 'psycopg2'" or "TypeError: 'module' object is not callable" when working with Python and SQLAlchemy.

If you are encountering a specific technical problem in Module 1, please let me know and I can try to find a solution in the FAQ.


In [154]:
for entry in final_response.candidates:
    contents.append(entry.content.parts[0])
    print(entry.content.parts[0])

    if entry.content.parts[0].function_call:      
        result = do_call(entry.content.parts[0].function_call)
        contents.append(result)
    elif entry.content.parts[0].text:
        print(entry.content.parts[0].text) 

video_metadata=None thought=None inline_data=None file_data=None thought_signature=b"\n\xe7\x05\x01T\xa8\\\xee\x08!5o \xc4U\xe1q\x8d@?\xa6j\xaa\xf9\xa9\x06\x1boUw_\xd4\xccM\x9f;\x15\xd6S\xd6\xe4\x97\x9c\x04\xab\xa9\xef\xea\xff\xc9j=*\xe6Bc\x06U\x13\x9e\x89\x8d\x9cb)\xf3\xa0:\xc8\x00\xa4Q:\xf7\xf0\xec|>\xdf\xdf\x83u\x92%#\x9e\x98\x82`7^3\xf6\xd9)\x1c\xc4\xc6\xd1\xeb\xcf\xa3c\x1d\xc7<\x83\xd7\x90\x1d\x9dK\x84u\x1f\x1c0\xe9\x82\xb0d\x1f-&\xbb\xb1E\x15H3\xfb\xf0TV(\xab\xeaYec\xe9\xa7\xb8\x9cy\xd0\x05\xf2\x19b*\xa2\xa7&\x8fJJ\xdc\xd9\xe0\x17\x1d\xaa\x96\xc9}g\xb0\xf3\x95\x98\xf6\xa6<\x9aq\x87\x9au\x83\xdbX\xd8\xe5juX\xd7\x9b\xfbU\xc5ZG\x02\x00\xb2\xa4z#\x18\n\x02\x90\xd0\\\xe2t\xd6mL\x01\xfbX\xbc\x8fd\xf4\xd1&\xe7f}\x7f'\xcb\xabnMf\xaePpm@j\xff\xa5\xe9\xeda\xe0\xdah\x10ZB2\xd3zGz\xff\xc6\xbf\xbaoA\xc3`\x05\xdb\x1b\x05\xf4}@_\x1e\x87\xba\xb3\x06\x993\xd2\xe1\xd8,\x8a;\x17?\x87\x0b\xbf\xb4F\xb6f\xa4\xbe\xa3\xee\x17\x88\xfa\x10$\xf8\x1b\xbf\x88q+\x83\x1f\x9c\x0b\xbep\xea\xac4%W\xdb\xe9jDx\xcf/

In [ ]:
contents

In [158]:
developer_prompt_template = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.
When using FAQ, perform deep topic exploration: make one request to FAQ,
and then based on the results, make more requests.

At the end of each response, ask the user a follow up question based on your answer.

<QUESTION>
{question}
</QUESTION>
""".strip()



In [157]:
chat_messages

["You're a course teaching assistant. \nYou're given a question from a course student and your task is to answer it.\n\nUse FAQ if your own knowledge is not sufficient to answer the question.\nWhen using FAQ, perform deep topic exploration: make one request to FAQ,\nand then based on the results, make more requests.\n\nAt the end of each response, ask the user a follow up question based on your answer.\n\n<QUESTION>\nwhat do I need to do to be successful at module 1?\n</QUESTION>"]

In [170]:
chat_messages = []
# Define the function declaration for the model
search_tool = {
    "name": "search",
    "description": "Searches the FAQ database.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ",
            }
        },
        "required": ["query"],
    },
}

tools = types.Tool(function_declarations=[search_tool])
config = types.GenerateContentConfig(tools=[tools])

while True: # main Q&A loop
    question = input() # How do I do my best for module 1?
    if question == 'stop':
        break

    chat_messages.append(developer_prompt_template.format(question=question))

    while True: # request-response loop - query API till get a message
        response = gemini_client.models.generate_content(
            model="gemini-2.5-flash",
            config=config,
            contents=chat_messages,
        )
        #print(response)

        has_messages = False

        for entry in response.candidates:
            for part in entry.content.parts:
                chat_messages.append(part)
            
                if part.function_call: 
                    print('\nfunction_call:', entry)
                    print()
                    result = do_call(part.function_call)
                    chat_messages.append(result)
                elif part.text == '\n':
                    print(part.text) 
                    has_messages = False
                elif part.text:
                    print(part.text) 
                    print()
                    has_messages = True

        if has_messages:
            break

 how do i do well in module 1?



function_call: content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=b'\n\xb5\x04\x01T\xa8\\\xee=\xeb\xef.\xe7B<\x87\xa2\xab$4\xceD\x03\xeeM\xddd\xfd\xb4\xf4;\xe7\xa0\xcbN/<\x91\xe3.,\xb8\xf8\xa1|\x02\xe8\xfc\x05\xb4\xeeD\xfd\x03\x9aV\xa0\x03\xce\x8d\x97\x9f\t\xf0\x85\xd19\x9b\xa0\xa7\xa3\xb19\xc3?\x93\xbb\x85\xb8\x80\xa7J\x15\x80\xd9{|\xdb\xb8\x92\x00fkH\x9b)\xa7\xc40\xbf8\xee\xef~3l\xdf\x0e\xd9J\xe3\xd4v`f\x83 \x88\xe6\xf3\x85\x8c\xbdM\xd6\xd8\xb2P\xaa\xd5i\xcaVJ\xc1\x99\x0e\x88\xe6\xf1\xf7\xa7N\xe0\xe4\x89\x0f\x81\xc3`\xd2\x86\xc4\x9c\xe8\x02\r3t\x1aQ\x0fx%\x90\xb0ic\x0b\xd1\x83e\xde\xb4\xae\xecOX5\x1fJLW\xb4(G\x05\x10\x07\x93M\xdeg\xbb\xc4\x965^\x8c?\x1c\x99\x1ak\x12I\xc4.\x1a\x87\xe7O\x1dG\xa7\xf5\xfa\xf4\x03\xbbX\xaff,a\xcf)w\x8d\xb2\x926\xef\xab\xc6|\x9d|\xaa\xd4<Jb\x93\x88\xf1\x83\xe1\x7f\x93\xf8#\xb4V\xbfX\x87o\xf2W\x1f#\xae\x91\x00)d\xe5z\xc4\x83\x16\x1d`\xef>\xb8\x9f[\xfb\x9a\xb4\xc0\xe0\xf1Q\xbby\xb8\xe9G}\x03Eq\

 I need help with docker





function_call: content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='\n'), Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=b'\n\xd1\x03\x01T\xa8\\\xee\x1f\xc9\\\xff\xc0\xc9\xc5\xeb=<\x9b\xdbtn\xc6V\xa5\xa5\xf3\xcb~x\xd2\x89\x0e\x02\xe6\xfd\x10a\t\x0b\x07\xb1\x87\x04c\x9c\xd4\xdfq\xb0A<On\xcb\xf2\xddy\xe5x\xff\xf8[\xe9\xa1m\xd1\x012\xaah\xb00|\x11F\xa0\xaf\xc3\xaf\xddO6\xd6(I\x9c\xecN\x13\xf4\x9fn\xfa\xf3&]\xb8\xca\xad\x11y\x8a\x08\xd1G\x0c\xae.\xec\xf2\x91\x84+\xc7\xb2m\xc7/\xaaX8F\xee\x12\xfd\x81\xc6Hj\xd4\xbeO\x1fE>t\xc7?\xe0\x1f\x0f\xfe\xe8\x90\xb7=\xb6s\x14;S\xb9\xde\x1e,g\x96H_\x87:3PR\xbfx\xc0\x925(\xfd\x01P\xf2\xee\xc9\xbf\xcde;|\x07k\x13\x96X\x13\xb0\r\xb9\xfa\x10\xbai*\xc8a\x89\xc1\xe1\xcf\xc6%*\x07K\x9f<\xc2\x13\xdf\x11\xfeI\xab\xba\xe2\xf4\xff\xca\xaf\xba\xc7\x89\x

 I need help starting docker





function_call: content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='\n'), Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=b'\n\xbe\x02\x01T\xa8\\\xee\x80\xc1S\x93\x85RUn\x9d\x89\xab\x7f\xb5\xc9,uF\x01\xf2\xfb\xfd|\x98&3\xc2\xb0\\\xb0H,czl\xa7\x14\x7f\xc7\x8a0\x9d\xc8W\x88\x113\xcc=\xeb\xc3\xf04\x9f\xda\xfe\xbeM&\xbf\x03\xf2\xfd"\x01\x9a\xd1\xe9\x0b\xa9)#\xee\xb8\xff\x01I\x99\x07\xdd\x03\xd4\x91\xab\xd8*u\xccq\xd9\xc4\xcd\t\xb4H\x99\x19Z!\xd3WGztN\x9b7\xe3I%\xc1\xc1\x9a\xf2\xde\xbdv\x19\xff\x02\xc4[\x82kY\x90\xdd\xc8m\xf3\xef\xf0tX\xa1\x9an\xb6\x00\xe9\x15Cm\t\xd9\x1f\xb7\xc5\xec\xd72 1\xad\x99\xa1\x1f\x90\xc8^\x10w\xee\xd4_cI\xc5\xc1 \xdd\xec4\x0f9\xc5u\x14\xf5\x15\x12\xa6\xc8\xb4\xcd\xe2V\xf9Cr\xe8\x05\x7fs\xbaR_p\x17* !\xd5\xa0\x06\xe5i\xb1\'\xb3\xc5\n0\x0exZO\xbb\x82\xd6\

 stop


In [ ]:
chat_messages

In [171]:
import chat_assistant

ModuleNotFoundError: No module named 'markdown'

In [172]:
!pip install markdown


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [224]:
import chat_assistant6

In [203]:
search_tool

{'name': 'search',
 'description': 'Searches the FAQ database.',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string',
    'description': 'Search query text to look up in the course FAQ'}},
  'required': ['query']}}

In [225]:
tools = chat_assistant6.Tools()
tools.add_tool(search, search_tool)

In [213]:
tools.get_tools()


[{'name': 'search',
  'description': 'Searches the FAQ database.',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'Search query text to look up in the course FAQ'}},
   'required': ['query']}}]

In [214]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.

<QUESTION>
{question}
</QUESTION>
""".strip()

In [226]:
chat_interface = chat_assistant6.ChatInterface()

In [227]:
chat = chat_assistant6.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=gemini_client
)

In [ ]:
chat.run()

You: how do i do well in module 1?


You: please search the fax on the information how to do well in module 1
